#### Conectando Python com API do Google Sheets

In [1]:
import pandas as pd
import gspread
import json
import os
import warnings
import unidecode
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()
key_json = os.getenv('KEY_JSON')

# Converte a string JSON em um dicionário Python
google_client = gspread.service_account_from_dict(json.loads(key_json))

# Identificador único da planilha do Google Sheets
SHEET_ID = '1tbJh_GQol0Ax5uB-AjwI26yW9piCIp0peuKV1N5nMeQ'

# Abre a planilha do Google Sheets usando o código único da planilha
spreadsheet = google_client.open_by_key(SHEET_ID)

# Seleciona a aba chamada 'Dados' dentro da planilha
worksheet = spreadsheet.worksheet('db')

# Obtém todos os valores da worksheet (retorna uma lista de listas)
dados = worksheet.get_all_values()

# A primeira linha é usada como o cabeçalho (nomes das colunas)
df = pd.DataFrame(dados[1:], columns=dados[0])

#### Limpeza e Preparação dos dados

In [2]:
# Padronizando o nome das colunas
df.rename(columns=str.lower, inplace=True)
df.columns = df.columns.str.replace(' ', '_')

# Removendo os acentos das colunas
df.columns = [unidecode.unidecode(col) for col in df.columns]

In [3]:
# Converter colunas de data/hora para o tipo datetime
df['chegada_cd'] = pd.to_datetime(df['chegada_cd'])
df['inicio_do_carregamento'] = pd.to_datetime(df['inicio_do_carregamento'])
df['fim_do_carregamento'] = pd.to_datetime(df['fim_do_carregamento'])
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

# Lista de colunas que precisam ser convertidas para timedelta
timedelta_columns = [
    'tempo_de_espera', 
    'tempo_de_expedicao', 
    'tempo_total', 
    'hora_chegada_cd', 
    'hora_inicio_do_carregamento', 
    'hora_fim_do_carregamento'
]

# Aplicar a conversão para timedelta usando apply
df[timedelta_columns] = df[timedelta_columns].apply(pd.to_timedelta)

# Ajustar "quantidade_remessas" como tipo numérico (inteiro)
df['quantidade_remessas'] = pd.to_numeric(df['quantidade_remessas'], errors='coerce')

In [ ]:
df.info()

#### Transformando os dados com `groupby`

In [ ]:
 # Tempo Médio de Expedição por Data
tempo_medio_dia = df.groupby(['data', 'unidade']).agg(
                    media_tempo_espera = pd.NamedAgg('tempo_de_espera', 'mean'),
                    media_tempo_expedicao = pd.NamedAgg('tempo_de_expedicao', 'mean'),
                    media_tempo_total = pd.NamedAgg('tempo_total', 'mean'),
                    total_remessas = pd.NamedAgg('quantidade_remessas', 'sum'),
                    total_motoristas = pd.NamedAgg('nome', 'count')).reset_index()

def format_timedelta(td):
    # Formata como HH:MM:SS
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Aplicar a formatação das médias
tempo_medio_dia[['media_tempo_espera',
                 'media_tempo_expedicao',
                 'media_tempo_total']] = tempo_medio_dia[['media_tempo_espera',
                                                         'media_tempo_expedicao',
                                                         'media_tempo_total']].applymap(format_timedelta)

tempo_medio_dia

In [6]:
tempo_medio_dia.to_clipboard(index=False)